In [23]:
using Revise
using Distributions, LinearAlgebra, StatsBase
using Test

In [24]:
includet("msvar_temp.jl")


In [25]:
St = [.9, .1]
St1 = [.8, .2]
St1T = [.85, .15]

transition_matrix = [.9 .1;.2 .8]

r = transition_matrix * (St1T ./ St1) .* St



2-element Vector{Float64}:
 0.928125
 0.08125

In [26]:
n_samples = 10_000
regimes = sample([1,2], ProbabilityWeights([0.8, 0.2]), n_samples)

dist1 = Normal(.13,.08)
dist2 = Normal(-.03, .2)


samples = zeros(n_samples)

for s in 1:n_samples
    if regimes[s] == 1
        samples[s] = rand(dist1)
    else 
        samples[s] = rand(dist2)
    end
end

display(mean(samples))
display(std(samples))
display(skewness(samples))
display(kurtosis(samples))



0.09653642740550505

0.1332994430456781

-1.286506869693468

3.1989323604980395

In [27]:
mu = [0.8, 0.2]' * [.13, -.03]
sig = sqrt([0.8, 0.2]' * [.08^2, .2^2])

samples = rand(Normal(mu, sig), n_samples)

display(mean(samples))
display(std(samples))
display(skewness(samples))
display(kurtosis(samples))

0.0991838190796853

0.1148074393740439

-0.017941337482077345

0.007729280858885357

In [28]:
var = .8(1-.8)*(.13+.03)^2 + .8 * .08^2 + (1-.8) * .2^2

sqrt(var)

0.1312097557348538

Simulate MSVAR TODO 
- first period simulate regimes
- simulate returns for every scenarios
- for next periods:
  - simulate regime based on current regime and transition matrix
  - siumlate returns based on regime

Two regimes, tree regimes, constand means, coeff, sigmas.



In [74]:
X = [.12, .06]

transition_matrix = [.9 .1;.2 .8]

states_zero = [.67, .33]


Β1 = [.08 .3 .2; .03 .1 .6]
Σ1 = cor2cov([1 .3; .3 1], [.15, .03])
Β2 = [.085 .35 .25; .035 .15 .63]
Σ2 = cor2cov([1 .35; .35 1], [.16, .035])
 

regimes = simulate_next_regimes(states_zero, transition_matrix, 2_000)

result = simulate_returns_step(regimes, [Β1, Β2], [Σ1, Σ2], repeat(X, 1, 2_000))

@test mean(result, dims=2) ≈ [.132, .081] atol=.009



Test Passed

In [66]:
n_scenarios = 10

X1 = add_intercept(repeat(X, 1, n_scenarios), n_scenarios)

3×10 Matrix{Float64}:
  1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0
 12.0   12.0   12.0   12.0   12.0   12.0   12.0   12.0   12.0   12.0
  0.06   0.06   0.06   0.06   0.06   0.06   0.06   0.06   0.06   0.06

In [56]:
repeat(X, 1, n_scenarios)

3×10 Matrix{Float64}:
  1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0
 12.0   12.0   12.0   12.0   12.0   12.0   12.0   12.0   12.0   12.0
  0.06   0.06   0.06   0.06   0.06   0.06   0.06   0.06   0.06   0.06

In [81]:
result1 = simulate_msvar(transition_matrix, [Β1, Β1], [Σ1, Σ2], X, states_zero, 3, 2_000)
@test mean(result1[:,1,:], dims=2) ≈ [.132, .081] atol=.009


Test Passed

In [82]:
display(mean(result1[1,1,:]))
display(std(result1[1,1,:]))
display(skewness(result1[1,1,:]))
display(kurtosis(result1[1,1,:]))

0.12387482174426774

0.15281185857514196

-0.16752245605786467

-0.102533382428458

In [97]:
regimes2 = simulate_next_regimes_for_previous_regimes(regimes, transition_matrix)

expected_n_regime_one = (0.7 * 0.9 + 0.3 * 0.2) * 2000
@test count(i -> i == 1, regimes2 ) ≈ 1380 atol=80

Test Passed

In [103]:
result = simulate_msvar(transition_matrix, [Β1, Β1], [Σ1, Σ2], X, states_zero, 3, 2_000)

mean(result[:,2,:],dims=2)

2×1 Matrix{Float64}:
 0.14040112420881062
 0.09127847758781459

In [102]:
result = simulate_msvar(transition_matrix, [Β1, Β1], [Σ1, Σ2], X, states_zero, 3, 2_000)

mean(result[:,3,:],dims=2)

2×1 Matrix{Float64}:
 0.1418263614435427
 0.09559648120958708

In [127]:
X = [.12, .06]

transition_matrix = [.9 .1;.2 .8]

states_zero = [.67, .33]


Β1 = [.09 .3 .2; .03 .1 .6]
Σ1 = cor2cov([1 .3; .3 1], [.08, .03])
Β2 = [-.02 .35 .25; .035 .15 .63]
Σ2 = cor2cov([1 .35; .35 1], [.18, .035])

result = simulate_msvar(transition_matrix, [Β1, Β1], [Σ1, Σ2], X, states_zero, 120, 100_000)

display("T1")
display(mean(result[1,1,:]))
display(std(result[1,1,:]))
display(skewness(result[1,1,:]))
display(kurtosis(result[1,1,:]))

display("T10")
display(mean(result[1,10,:]))
display(std(result[1,10,:]))
display(skewness(result[1,10,:]))
display(kurtosis(result[1,10,:]))

"T1"

0.1380626226539845

0.12238611858949694

0.0009790117773336674

2.004145070240231

"T10"

0.1614993251735118

0.13049666936501517

0.010229008814393752

1.8164161345910825

expected mean 
for regime 1 
prob regime one, previous regime one mean, 
prob regime 2 to 1, previous regime two mean

In [38]:
0.669 * 300 

200.70000000000002

In [39]:
Β1 = [.08 .3 .2; .03 .1 .6]'
X = [1. .12 .06]

X * Β1

1×2 Matrix{Float64}:
 0.128  0.078

In [40]:
Β2 = [.085 .35 .25; .035 .15 .63]'
Σ2 = cor2cov([1 .35; .35 1], [.16, .035])

X * Β2

1×2 Matrix{Float64}:
 0.142  0.0908

In [41]:
.7 * .128 + .3 * .142 

0.13219999999999998

In [42]:
.7 * .078 + .3 * .0908

0.08184

In [43]:
.08 * 1 + .3 * .12 + .2 * .06

0.128

In [44]:
mean([.1 .2; .3 .4], dims=2)

2×1 Matrix{Float64}:
 0.15000000000000002
 0.35

In [134]:
Σ1

2×2 Matrix{Float64}:
 0.0064   0.00072
 0.00072  0.0009

In [132]:
cholesky(Σ1).L 

2×2 LowerTriangular{Float64, Matrix{Float64}}:
 0.08    ⋅ 
 0.009  0.0286182